In [ ]:
!mkdir -p ~/agave/funwave-tvd-docker

%cd ~/agave

!pip3 install setvar

import re
import os
import sys
from setvar import *
from time import sleep

# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
loadvar()
!auth-tokens-refresh

<h2>Creating the Docker Image</h2>
To start, we need a Dockerfile, which has a number of simple commands.
It starts with "FROM" which can specify any docker image available from DockerHub. That not only includes basic operating systems such as "ubunto", "fedora", "centos", etc. but specialized containers made by anyone with a dockerhub account. I've provided "science-base" which has OpenMPI 2.1.1 and some standard compilers, i.e. gfortran, gcc, and g++.

MAINTAINER is a bit of metadata that (hopefully) will allow you to contact the container's creator, if need be.

WORKDIR is the dockerfile equivalent of the "cd" command. Note that running "cd" will not change your directory.

RUN simply runs the command that follows. Because the container is saved after each step, we want to avoid creating files that we don't want to keep (we want containers to be as small as possible).

USER specifies the user id for running subsequent RUN commands.

COPY can be used to copy files into the container from the build directory.

ENTRYPOINT is a script that runs when the container starts up. What our script does is create a new user on the docker image with a user id and name that is convenient.

In [ ]:
writefile("funwave-tvd-docker/Dockerfile","""
FROM stevenrbrandt/science-base

MAINTAINER Steven R. Brandt <sbrandt@cct.lsu.edu>
RUN cd && \
    git clone https://github.com/fengyanshi/FUNWAVE-TVD && \
    cd FUNWAVE-TVD/src && \
    perl -p -i -e 's/FLAG_8 = -DCOUPLING/#$&/' Makefile && \
    make

WORKDIR /home/jovyan/FUNWAVE-TVD/src
RUN mkdir -p /home/jovyan/FUNWAVE-TVD/rundir
WORKDIR /home/jovyan/FUNWAVE-TVD/rundir
USER root
COPY entrypoint.sh /usr/local/bin
RUN chmod 755 /usr/local/bin/entrypoint.sh
ENTRYPOINT ["/usr/local/bin/entrypoint.sh"]
""")

This is our entrypoint file. It can be any bash script, and it is not required.

In [ ]:
writefile("funwave-tvd-docker/entrypoint.sh","""
#!/bin/bash
echo ARGS: \$@
/home/jovyan/FUNWAVE-TVD/src/funwave_vessel "\$@"
""")

Now that we've create our Dockerfile and entrypoint.sh, bundle them up in a tarball and send them somewhere that agave can access them.

In [ ]:
!tar -czf dockerjob.tgz -C funwave-tvd-docker Dockerfile entrypoint.sh
!files-mkdir -S ${STORAGE_MACHINE} -N funwave-tvd-docker
!files-upload -F dockerjob.tgz -S ${STORAGE_MACHINE} funwave-tvd-docker/

In [ ]:
import runagavecmd as r

Run the docker build command. We will "tag" this build with the name "funwave-tvd" when it is complete.

In [ ]:
r.runagavecmd(
    "tar xzf dockerjob.tgz && sudo docker build --rm -t funwave-tvd .",
    "agave://${STORAGE_MACHINE}/funwave-tvd-docker/dockerjob.tgz"
)

In [ ]:
!jobs-output-get ${JOB_ID} fork-command-1.err
!cat fork-command-1.err

<h2>Running the Docker Image</h2>
It is possible to run docker interactively, but that isn't convenient inside scripts. So instead, we start it in detached mode, with the -d flag.

Because your docker image has its own internal file system, it can't see files on the host machine. You can, however, transfer them using the "docker cp" command.

Running docker is slightly tricky. When a Docker image starts up, you can execute any command you want--but when you type "exit" all the changes you've made to the file system vanish. Therefore it's necessary to copy them out before the docker container stops.

In [ ]:
writefile("rundock.sh","""
rm -fr cid.txt out.tgz

# Start a docker image running in detached mode, write the container id to cid.txt
sudo docker run -d -it --rm --cidfile cid.txt -e USER_ID=$(id -u) -e USER_NAME=$USER stevenrbrandt/funwave-tvd bash

# Store the container id in CID for convenience
CID=\$(cat cid.txt)

# Copy the input.txt file into the running image
sudo docker cp input.txt \$CID:/home/jovyan/FUNWAVE-TVD/rundir/

# Run funwave on the image
sudo docker exec --user jovyan \$CID mpirun -np 4 /home/jovyan/FUNWAVE-TVD/src/funwave_vessel

# Extract the output files from the running image
# Having them in a tgz makes it more convenient to fetch them with jobs-output-get
sudo docker exec --user jovyan \$CID tar czf -  output > out.tgz

# Stop the image
sudo docker stop \$CID

# List the output files
tar tzf out.tgz
""")

Upload the input.txt file and the rundock.sh script.

In [ ]:
!tar czf rundock.tgz rundock.sh input.txt
!files-upload -F rundock.tgz -S ${STORAGE_MACHINE} funwave-tvd-docker/

Execute the rundock.sh script

In [ ]:
r.runagavecmd(
    "tar xzf rundock.tgz && bash rundock.sh",
    "agave://${STORAGE_MACHINE}/funwave-tvd-docker/rundock.tgz")

Get the output of the job back to our local machine

In [ ]:
!jobs-output-list ${JOB_ID}
!jobs-output-get ${JOB_ID} out.tgz
!tar xzvf out.tgz

In [ ]:
!head output/eta_00010

<h2>Running with Singularity</h2>
If we have a public docker image, we can run it directly with Singularity. Singularity is desiged to be more HPC friendly than Docker. First, because it doesn't all the running user to access any user id but their own inside the container, and second, because singularity images can be run through MPI, making it easier to scale up to a distributed cluser.

In this first step, we build the singularity installation. Because the result of this job is intended to be an installation for subsequent jobs, we install it to a hard-coded directory rather than using the normal Agave job directory.

In [ ]:
!files-mkdir -S ${STORAGE_MACHINE} -N sing
!files-upload -F input.txt -S ${STORAGE_MACHINE} sing/
r.runagavecmd(
            "mkdir -p ~/singu && "+
            "cd ~/singu && "+
            "rm -f funwave-tvd.img && "+
            "singularity create funwave-tvd.img --size 2000 && "+
            "singularity import funwave-tvd.img docker://stevenrbrandt/funwave-tvd:latest")

Now that the Singularity image is built, we can run it with mpi. Notice that mpi executes the singularity command. The tricky part here is to make sure you've got the same version of mpi running inside and outside the container.

In [ ]:
!files-upload -F input.txt -S ${STORAGE_MACHINE} ./
r.runagavecmd(
    "export LD_LIBRARY_PATH=/usr/local/lib && "+
    "mpirun -np 4 singularity exec ~/singu/funwave-tvd.img /home/jovyan/FUNWAVE-TVD/src/funwave_vessel && "+
    "tar cvzf singout.tgz output",
    "agave://${STORAGE_MACHINE}/input.txt"
)

In [ ]:
!jobs-output-get ${JOB_ID} singout.tgz
!rm -fr output
!tar xzf singout.tgz

In [ ]:
!head output/v_00010